In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data/car_data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   2021 non-null   int64  
 1   car_id               2021 non-null   object 
 2   make                 2021 non-null   object 
 3   model                2021 non-null   object 
 4   year_of_manufacture  2021 non-null   int64  
 5   color                2021 non-null   object 
 6   condition            2021 non-null   object 
 7   mileage              1969 non-null   float64
 8   engine_size          1978 non-null   float64
 9   registered_city      1160 non-null   object 
 10  selling_condition    2020 non-null   object 
 11  bought_condition     2020 non-null   object 
 12  city                 2021 non-null   object 
 13  price                2021 non-null   int64  
 14  body_type            954 non-null    object 
 15  fuel_type            2020 non-null   o

In [4]:
data.drop(['id', 'car_id'], axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make                 2021 non-null   object 
 1   model                2021 non-null   object 
 2   year_of_manufacture  2021 non-null   int64  
 3   color                2021 non-null   object 
 4   condition            2021 non-null   object 
 5   mileage              1969 non-null   float64
 6   engine_size          1978 non-null   float64
 7   registered_city      1160 non-null   object 
 8   selling_condition    2020 non-null   object 
 9   bought_condition     2020 non-null   object 
 10  city                 2021 non-null   object 
 11  price                2021 non-null   int64  
 12  body_type            954 non-null    object 
 13  fuel_type            2020 non-null   object 
 14  transmission         2021 non-null   object 
dtypes: float64(2), int64(2), object(11)
me

In [7]:
car_data = data.copy()

Preprocessing

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [10]:
# feature and target variables
X = car_data.drop(columns = ["price"], axis=1)
y = car_data["price"]

In [16]:
# preprocessing numerical features
numerical_features = ["mileage", "engine_size"]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [18]:
# preprocessing categorical features
categorical_features = ["make", "model", "color", "condition", "registered_city", "selling_condition", "bought_condition", "city", "body_type", "fuel_type", "transmission"]


categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [19]:
# combining steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

Model

In [28]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [42]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(random_state = 42))])


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_size']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['make', 'model', 'color',
                                                   'condition',
                                                   'registered_city',
                                                   'selling_condition',
                                                   'bought_condition', 'city',
                                                   'body_type', 'fuel_type',
                                                   'transmission'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [45]:
y_pred_1 = model.predict(X_test)

Evaluation

In [46]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [47]:
mae = mean_absolute_error(y_test, y_pred_1)
mse = mean_squared_error(y_test, y_pred_1)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_1)

print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", rmse)
print("R2 Score: ", r2)


Mean Absolute Error:  4850596.547185185
Mean Squared Error:  136570427511982.73
Root Mean Squared Error:  11686335.076146958
R2 Score:  0.5019174099586886


Tuning

In [48]:
from sklearn.model_selection import GridSearchCV

In [49]:
param_grid = {
    'regressor__n_estimators' : [100, 200, 300],
    'regressor__max_depth' : [None, 10, 20, 30],
}

In [50]:
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=1, scoring="neg_mean_absolute_error")

In [51]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['mileage',
                                                                          'engine_size']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['make',
                                                                          'model',
                                                                          'color',
                                                                          'condition',
                                                                          'registered_city',
                                                                          'selling_condition',
                                                                          'bought_condition',
                                                                          'city',
                                                                          'body_type',
                                                                          'fuel_type',
                                                                          'transmission'])])),
                                       ('regressor',
                                        RandomForestRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'regressor__max_depth': [None, 10, 20, 30],
                         'regressor__n_estimators': [100, 200, 300]},
             scoring='neg_mean_absolute_error', verbose=1)

In [53]:
best_params = grid_search.best_params_
best_params

{'regressor__max_depth': 30, 'regressor__n_estimators': 200}

In [54]:
y_pred_2 = grid_search.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_2)
mse = mean_squared_error(y_test, y_pred_2)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_2)

print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", rmse)
print("R2 Score: ", r2)


Mean Absolute Error:  4897496.745666265
Mean Squared Error:  137757730432333.86
Root Mean Squared Error:  11737023.917174824
R2 Score:  0.49758722717676673


Saving model

In [55]:
import joblib

In [58]:
joblib.dump(grid_search, "model/car_prediction_model.pickle") 

['model/car_prediction_model.pickle']

: 